# Notebook 4 – Standard Model Mass Explorer

**Level:** Beginner  
**Topics:** Standard Model, fermion masses, Pocket_U Lite, screening, ξ, basic plotting

> **Provenance:** This notebook is based on:
> - `examples/pocket_u_mass_table.py`
> - `examples/pocket_u_geometry_showcase.py`

In [ ]:
# Optional: install deps if needed
# %pip install light-theory-realm pandas matplotlib

import pandas as pd
import matplotlib.pyplot as plt

from light_theory_realm import print_sm_table, get_particle_profile

# Optional: access list of particles from the submodule
try:
    from light_theory_realm.pocket_u_lite.sm_table import ALL_PARTICLES
except ImportError:
    ALL_PARTICLES = ["e", "mu", "tau", "u", "d", "s", "c", "b", "t"]

plt.rcParams["figure.figsize"] = (7, 5)

## 1. Raw Standard Model mass table

In [ ]:
print_sm_table()

## 2. Building a DataFrame of particle profiles

> Note: This notebook includes a small robustness patch for `compute_reeb_diagnostics` (safe to re-run; idempotent).


In [ ]:
# Monkeypatch for robustness (same as Notebook 1)
import light_theory_realm.pocket_u_lite.geometry as geom

# Only save the original once, even if you re-run this cell
if not hasattr(geom, "_orig_compute_reeb_diagnostics"):
    geom._orig_compute_reeb_diagnostics = geom.compute_reeb_diagnostics

def compute_reeb_diagnostics_fixed(primes, *args, **kwargs):
    """
    Wrapper around compute_reeb_diagnostics that ensures `primes` is a tuple,
    so expressions like `primes + (primes[0],)` work even if `primes` was a list.
    """
    primes = tuple(primes)  # list -> tuple (tuple -> tuple is also fine)
    return geom._orig_compute_reeb_diagnostics(primes, *args, **kwargs)

# Monkeypatch the module-level function used by get_particle_geometry
geom.compute_reeb_diagnostics = compute_reeb_diagnostics_fixed
print("Patched pocket_u_lite.geometry.compute_reeb_diagnostics (idempotent).")

profiles = []
for name in ALL_PARTICLES:
    p = get_particle_profile(name)
    profiles.append(p)

df = pd.DataFrame(profiles)
df

## 3. Predicted vs PDG mass

We’ll compare the Pocket_U Lite prediction `m_phys_MeV` against the PDG reference mass `pdg_mass_MeV` for each fermion.

In [ ]:
df_sorted = df.sort_values("pdg_mass_MeV")

names = df_sorted["name"]
m_pred = df_sorted["m_phys_MeV"]
m_pdg = df_sorted["pdg_mass_MeV"]

x = range(len(names))

plt.figure()
plt.bar(x, m_pdg, alpha=0.5, label="PDG mass")
plt.bar(x, m_pred, alpha=0.5, label="Pocket_U Lite")
plt.xticks(x, names)
plt.yscale("log")
plt.ylabel("Mass [MeV] (log scale)")
plt.title("Standard Model fermion masses: PDG vs Pocket_U Lite")
plt.legend()
plt.tight_layout()
plt.show()

## 4. Error vs PDG per particle

In [ ]:
plt.figure()
plt.bar(names, df_sorted["error_pct"])
plt.ylabel("Relative error [%]")
plt.title("Pocket_U Lite mass error vs PDG")
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()

## 5. Screening vs mass

Here we look at how the **screening percentage** correlates with mass. High screening ≈ strong “vacuum bubble”.

In [ ]:
plt.figure()
plt.scatter(df["m_phys_MeV"], df["screening_pct"])

for _, row in df.iterrows():
    plt.text(row["m_phys_MeV"], row["screening_pct"], row["name"], fontsize=8)

plt.xscale("log")
plt.xlabel("Predicted mass m_phys_MeV [MeV] (log)")
plt.ylabel("Screening [%]")
plt.title("Screening vs mass for Standard Model fermions")
plt.grid(True, which="both", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()

## 6. Case study: the electron

The electron is particularly interesting: its mass is tiny, and the model often predicts it is **almost completely screened** (screening ≈ 100%).

In [ ]:
e = df[df["name"] == "e"].iloc[0]

for key in ["m_phys_MeV", "pdg_mass_MeV", "error_pct", "screening_pct", "xi"]:
    print(f"{key:16s}: {e[key]}")

## 7. Next steps

- Try plotting **ξ vs mass** to see how the dark-energy–like resonance changes across particles.
- Compare leptons vs quarks by coloring the bars differently.
- Move on to:
  - **Notebook 5 – Koide Playground** for a deeper dive into the lepton mass pattern.
  - **Notebook 6 – Geometric Fingerprints of Particles** for Fisher, Berry, and ξ plots.